In [1]:
import numpy as np

## Schmidt decomposition demo

According to the Schmidt decomposition theorem, we should be able to write any state $|\psi\rangle$ as a sum over the tensor products between two (or more) vector spaces $|u_i\rangle, |v_i\rangle$, weighted by the Schmidt coefficients $\alpha_i$ according to

$$
    |\psi\rangle = \sum_{i = 1}^m \alpha_i |u_i\rangle \otimes |v_i\rangle.
$$

#### Example: the GHZ state
Let's try this for a simple example. Suppose we have a system with $N$ spin-1/2 particles that is in the GHZ state
$$
    |\psi\rangle = |\downarrow \downarrow \cdots \downarrow \rangle + |\uparrow \uparrow \cdots \uparrow \rangle.
$$
It lives in a $2^N$ dimensional Hilbert space, which we can construct as

$$
    \left\{ |\downarrow\rangle, |\uparrow\rangle \right\} \otimes \cdots \otimes \left\{ |\downarrow\rangle, |\uparrow\rangle \right\} = \left\{ |\downarrow\rangle, |\uparrow\rangle \right\}^{\otimes N}.
$$

I hope it doesn't need too much convincing that in this basis,

$$
    |\psi\rangle = \left( \begin{array}{} 1 \\ 0 \\ \vdots \\ 0 \\ 1 \end{array} \right)
$$

This is how we'd usually write it, but we can only do a Schmidt decomposition on a matrix. The number of rows in this matrix must be equal to the dimension of the left-hand subspace, and the number of columns must be equal to the dimension of the right-hand subspace. Effectively this means we 'bundle' some of the indices together. As long as we're consistent in the way we reshape and enumerate indices this will not lead to problems. (Numpy, by default, [uses C-like ordering](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html), with the last index changing fastest.)

Let's work out the example of four spins in the GHZ state. We'd like to cleave this system right in the middle. What does the SVD look like?

In [2]:
N = 4                        # Total number of spins
psi = np.zeros(2**N)         # Configure state in vector form
psi[0] = 1
psi[-1] = 1

Nleft = 2                    # Number of sites left of the cleaving point
Nright = N - Nleft           # Number of sites right of the cleaving point
psi_mat = np.reshape(psi, (2**Nleft, 2**Nright))
print("psi_mat = \n", psi_mat)

psi_mat = 
 [[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]


Our state is now described by a matrix. We can apply the SVD to this to decompose the state in a left- and a right-hand-side.

In [3]:
X, Y, Zt = np.linalg.svd(psi_mat)
print("LHS = \n", X)
print("RHS = \n", Zt.T, "\n")
print("Singular values = \n", Y)

LHS = 
 [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
RHS = 
 [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]] 

Singular values = 
 [1. 1. 0. 0.]


The left- and right-hand side bases are spanned by the _columns_ of the matrices printed here. The ordering may seem a little off, but they are sorted according to the size of their corresponding singular values. Colloquially speaking: the higher a singular value, the more important the corresponding bases states are in the representation of the object (here: state). 

For this example, we can see that on both the left- and right-hand side the 'most important' bases states are $(1 0 0 0)^T$ and $(0 0 0 1)^T$. In the two-particle basis that describes both the left- and the right-hand-side these are the states $|\downarrow\downarrow\rangle$ and $|\uparrow\uparrow\rangle$, which makes sense.

What's interesting is that only two of the four singular values are nonzero. This means that our full $2^N$-dimensional basis is too large to describe the system, and that we can get away with fewer states. We can compress everything by shaving off two of the basis vectors of `X` and `Z`, and two singular values off of `Y`.

In [5]:
np.dot( X[:, 0:2], np.dot(np.diag(Y[0:2]), Zt[0:2, :]) )

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])

And we get the same state! While this may not look so impressive here, this is actually the cornerstone of many modern numerical techniques such as TEBD and DMRG. It will enable us to describe some states that live in huge Hilbert spaces using a fraction of the numbers we would require otherwise, making some simulations an option.